In [3]:
import pandas as pd
import pyodbc
import sqlite3

In [ ]:
DB = {"servername": "localhost\SQLEXPRESS",
      "database": "greatoutdoors"}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                             ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')

export_cursor = export_conn.cursor()


# Load utilities


# Load tables

sales_db = sqlite3.connect("Databases/GreatOutdoors/go_sales.sqlite")
product = pd.read_sql_query("SELECT * FROM product;", sales_db)
country = pd.read_sql_query("SELECT * FROM country;", sales_db)
product_line = pd.read_sql_query("SELECT * FROM product_line;", sales_db)

sales_branch = pd.read_sql_query("SELECT * FROM sales_branch;", sales_db)
retailer_site = pd.read_sql_query("SELECT * FROM retailer_site;", sales_db)
product_type = pd.read_sql_query("SELECT * FROM product_type;", sales_db)
returns = pd.read_sql_query("SELECT * FROM returned_item;", sales_db)
order_details = pd.read_sql_query("SELECT * FROM order_details;", sales_db)
sales_staff = pd.read_sql_query("SELECT * FROM sales_staff;", sales_db)
order_header = pd.read_sql_query("SELECT * FROM order_header;", sales_db)
returned_item  = pd.read_sql_query("SELECT * FROM returned_item;", sales_db)
return_reason  = pd.read_sql_query("SELECT * FROM return_reason;", sales_db)
target = pd.read_sql_query("SELECT * FROM SALES_TARGETData;", sales_db)
order_method = pd.read_sql_query("SELECT * FROM order_method;", sales_db)



for row in export_cursor.tables():
    print(row.table_name)


In [ ]:
# Product


transformed_product = []

transformed_product = product.merge(product_type, 'inner', 'PRODUCT_TYPE_CODE').merge(product_line, 'inner', 'PRODUCT_LINE_CODE').rename(columns={
    "INTRODUCTION_DATE": "INTRODUCTION_DATE_date",
    "PRODUCT_TYPE_CODE": "PRODUCT_TYPE_CODE",
    "PRODUCTION_COST": "PRODUCTION_COST_number",
    "MARGIN": "MARGIN_NUMBER",
    "PRODUCT_IMAGE": "PRODUCT_IMAGE_image",
    "LANGUAGE": "LANGUAGE_language",
    "PRODUCT_TYPE_EN": "PRODUCT_TYPE_name",
    "DESCRIPTION": "DESCRIPTION_description",
}).drop(columns="TRIAL888_x", axis=1).drop(columns="TRIAL888_y", axis=1).drop(columns="TRIAL888", axis=1)


for index, row in transformed_product.iterrows():
    try:
        query = f"INSERT INTO Product VALUES ({row['PRODUCT_NUMBER']},  '{row['PRODUCT_NAME']}', '{ convert_date_format(row['INTRODUCTION_DATE_date']) }', {row['PRODUCT_TYPE_CODE']}, '{row['PRODUCT_TYPE_name']}', {row['PRODUCT_LINE_CODE']}, '{row['PRODUCT_LINE_EN']}', {row['PRODUCTION_COST_number']}, {row['MARGIN_NUMBER']},'{row['PRODUCT_IMAGE_image']}', '{row['LANGUAGE_language']}', '{escape_single_quotes(row['DESCRIPTION_description'])}');"
        export_cursor.execute(query)
        print(query)
    except pyodbc.Error as e:
        print(query)
        # print(e)

    export_cursor.commit()


In [ ]:
# Order


merged_df = pd.merge(order_header, order_details, on='ORDER_NUMBER', how='inner')
order = pd.merge(merged_df, returned_item, on='ORDER_DETAIL_CODE', how = 'inner')

for index, row in order.iterrows():
    try:
        query = f"INSERT INTO 'orders' VALUES ({row['ORDER_NUMBER']},  '{escape_single_quotes(row['RETAILER_NAME'])}', '{row['ORDER_DATE']}', 1, 1, 2024, {row['ORDER_DETAIL_CODE']}, {row['PRODUCT_NUMBER']}, {row['QUANTITY']}, {row['UNIT_COST']}, {row['UNIT_PRICE']}, {row['UNIT_SALE_PRICE']}, {row['RETURN_CODE']}, '{convert_date_format(strip_time_from_string(row['RETURN_DATE']))}', {row['RETURN_REASON_CODE']}, {row['RETURN_QUANTITY']}, {row['ORDER_METHOD_CODE']}, {row['SALES_STAFF_CODE']}, {row['SALES_BRANCH_CODE']}, {row['RETAILER_SITE_CODE']});"
        print(query)
    except pyodbc.Error as e:
        print(query)
        print(e)
        
order

In [ ]:
# Sales Forecast & Inventory

sales_data_forecast = pd.read_csv("Databases/GreatOutdoors/GO_SALES_PRODUCT_FORECASTData.csv")
sales_inventory_levels = pd.read_csv("Databases/GreatOutdoors/GO_SALES_INVENTORY_LEVELSData.csv")

sales_data_merged = pd.merge(sales_data_forecast, sales_inventory_levels, 
                     left_on=['PRODUCT_NUMBER','YEAR','MONTH'],
                     right_on=['PRODUCT_NUMBER','INVENTORY_YEAR','INVENTORY_MONTH'],
                     how='inner')



sales_data_merged = sales_data_merged.rename(columns={'YEAR': 'FORECAST_YEAR', 'MONTH': 'FORECAST_MONTH'})



for index, row in sales_data_merged.iterrows():
    try:
        query = f"INSERT INTO 'SALESDATA' VALUES ({row['PRODUCT_NUMBER']}, {row['FORECAST_MONTH']}, {row['FORECAST_YEAR']}, {row['INVENTORY_MONTH']}, {row['INVENTORY_YEAR']}, {row['INVENTORY_COUNT']});"
        print(query)
    except pyodbc.Error as e:
        print(query)
        print(e)


# len(sales_data_merged)




In [ ]:
# Order Method

for index, row in order_method.iterrows():
    try:
        query = f"INSERT INTO order_method VALUES ({row['ORDER_METHOD_CODE']}, '{row['ORDER_METHOD_EN']}');"
        print(query)
    except pyodbc.Error as e:
        print(query)
        print(e)

In [34]:
#Retailer
crm_db = sqlite3.connect("Databases/GreatOutdoors/go_crm.sqlite")

retailer = pd.read_sql_query("SELECT * FROM retailer;", crm_db)
retailer_type = pd.read_sql_query("Select * FROM retailer_type", crm_db)
retailer_segment = pd.read_sql_query("select * from retailer_segment", crm_db)
retailer_headquarters = pd.read_sql_query("select * from retailer_headquarters", crm_db)

merged_df = pd.merge(retailer,retailer_type,on="RETAILER_TYPE_CODE",how="left")
merged_df2 = pd.merge(merged_df,retailer_type,on="RETAILER_TYPE_CODE", how="left")
merged_df3 = pd.merge(merged_df2,retailer_headquarters,on="RETAILER_CODEMR", how = "left")
retailer_segment = retailer_segment.rename(columns={'TRIAL222': 'TRIAL222_segment'})
merged_df4 = pd.merge(merged_df3,retailer_segment,on="SEGMENT_CODE",how="left")

merged_df4

,RETAILER_CODE,RETAILER_CODEMR,COMPANY_NAME,RETAILER_TYPE_CODE,TRIAL219,RETAILER_TYPE_EN_x,TRIAL222_x,RETAILER_TYPE_EN_y,TRIAL222_y,RETAILER_NAME,...,POSTAL_ZONE,COUNTRY_CODE,PHONE,FAX,SEGMENT_CODE,TRIAL222,LANGUAGE,SEGMENT_NAME,SEGMENT_DESCRIPTION,TRIAL222_segment
0,1,None,ActiForme,5,T,Equipment Rental Store,T,Equipment Rental Store,T,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100,70,Golf Masters,1,T,Golf Shop,T,Golf Shop,T,Golf Masters,...,68538,3,1 (402) 475-4317,1 (402) 475-4717,3,T,EN,Golf Shop,Primarily offers golf equipment and accessories.,T
2,101,71,The Marketplace,2,T,Department Store,T,Department Store,T,The Marketplace,...,10039,3,1 (212) 477-7132,1 (212) 477-9716,5,T,EN,Department Store,Offers a wide range of goods.,T
3,102,72,Camping Equipment Online,3,T,Direct Marketing,T,Direct Marketing,T,Camping Equipment Online,...,03761,3,1 (603) 448-9004,1 (603) 448-6412,7,T,EN,Direct Marketing,Offers a wide range of goods through mail orde...,T
4,103,73,Tamarack Outfitter Rentals,5,T,Equipment Rental Store,T,Equipment Rental Store,T,Tamarack Outfitter Rentals,...,97295,3,1 (503) 285-0954,1 (503) 285-0894,9,T,EN,Equipment Rental Store,Offers various sports and outdoors equipment f...,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,95,5,Falcon Outfitters,6,T,Outdoors Shop,T,Outdoors Shop,T,Falcon Outfitters,...,K1G 4Z0,4,1 (613) 741-5538,1 (613) 740-6701,1,T,EN,Outdoors Shop - Exclusive,"Primarily offers camping supplies, hiking equi...",T
105,96,6,Maximum Sports,8,T,Sports Store,T,Sports Store,T,Maximum Sports,...,M8Y 4K8,4,1 (416) 495-1223,1 (416) 491-8738,12,T,EN,Sports Store,Primarily offers a wide range of sporting goods.,T
106,97,7,Act'N'Up Fitness,4,T,Warehouse Store,T,Warehouse Store,T,Act'N'Up Fitness,...,M2P 4F6,4,1 (416) 496-9093,1 (416) 496-5066,8,T,EN,Warehouse Store,Offers a wide range of goods in a large wareho...,T
107,98,None,Advanced Climbing Ltd,5,T,Equipment Rental Store,T,Equipment Rental Store,T,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
sql_column_names = ['RETAILER_code', 'RETAILER_company', 'column3']
merged_df4_reordered = merged_df4[sql_column_names]
merged_df4_reordered

,RETAILER_CODE,RETAILER_CODEMR,RETAILER_company_name,RETAILER_TYPE_CODE,RETAILER_TYPE_name,RETAILER_NAME,ADDRESS1,ADDRESS2,CITY,REGION,COUNTRY_CODE,SEGMENT_CODE,LANGUAGE,SEGMENT_NAME,SEGMENT_DESCRIPTION
0,1,None,ActiForme,5,Equipment Rental Store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100,70,Golf Masters,1,Golf Shop,Golf Masters,2845 South Second Street,None,Lincoln,Nebraska,3,3,EN,Golf Shop,Primarily offers golf equipment and accessories.
2,101,71,The Marketplace,2,Department Store,The Marketplace,1902 Lancaster Street,None,New York,New York,3,5,EN,Department Store,Offers a wide range of goods.
3,102,72,Camping Equipment Online,3,Direct Marketing,Camping Equipment Online,332 South Main Street,P.O. Box 224,Hanover,New Hampshire,3,7,EN,Direct Marketing,Offers a wide range of goods through mail orde...
4,103,73,Tamarack Outfitter Rentals,5,Equipment Rental Store,Tamarack Outfitter Rentals,582 NE Tomahawk Island Drive,None,Portland,Oregon,3,9,EN,Equipment Rental Store,Offers various sports and outdoors equipment f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,95,5,Falcon Outfitters,6,Outdoors Shop,Falcon Outfitters,9755 Riverside Drive,None,Ottawa,Ontario,4,1,EN,Outdoors Shop - Exclusive,"Primarily offers camping supplies, hiking equi..."
105,96,6,Maximum Sports,8,Sports Store,Maximum Sports,"Suite 12, 801 Dundas Street",None,Toronto,Ontario,4,12,EN,Sports Store,Primarily offers a wide range of sporting goods.
106,97,7,Act'N'Up Fitness,4,Warehouse Store,Act'N'Up Fitness,"87, 2399 Sommerset Avenue",None,Toronto,Ontario,4,8,EN,Warehouse Store,Offers a wide range of goods in a large wareho...
107,98,None,Advanced Climbing Ltd,5,Equipment Rental Store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
